In [1]:
import os, re, copy, json, pprint, argparse, warnings, xmltojson, shutil
import pandas as pd
from pathlib import Path
import bibtexparser
warnings.filterwarnings('ignore')

In [ ]:
with open('../financialanomalies/references.bib') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

for item in bib_database.entries:
    print(item.get('title'))


In [ ]:
with open("financial_anomalies_cited_network_raw", "r") as fp:
    academic_networks = json.load(fp)
counter = 0
for item in academic_networks:
    if not isinstance(item.get('main_title'), type(None)) | isinstance(item.get('cited_titles'), type(None)):
        counter +=  1
        print(counter, item.get('main_title'),item.get('main_authors'))
        #, len(item.get('cited_titles')))

In [ ]:
!clear
!git add .
!git status
!git commit -m "citations graph"
!git push
!clear 
